# Gemma_re: Project Z

Description: Gemma7b finetuned for relation classificaion tasks


In [14]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

# 0. Import Libraries

In [15]:
from datetime import datetime 
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from datasets import load_dataset
from peft import LoraConfig
from trl import SFTTrainer
from huggingface_hub import login
login('YOURTOKENHERE')

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# 1. Load Model and tokenizer 

In [16]:
model_id = "google/gemma-7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [17]:
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir='/root/autodl-tmp/Projects/HuggingFaceCache')
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    quantization_config=bnb_config,
    #attn_implementation="flash_attention_2",
    cache_dir='/root/autodl-tmp/Projects/HuggingFaceCache/'
    )

`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.46s/it]


# 3. Load train and val data

In [18]:
train_dataset = load_dataset("json", data_files="/root/autodl-tmp/Projects/Gemma_RE/gemma_re_z/semeval/semeval_train_processed_cloze.json", split="train")
eval_dataset = load_dataset("json", data_files="/root/autodl-tmp/Projects/Gemma_RE/gemma_re_z/semeval/semeval_val_processed_cloze.json", split="train")

# 4. Training arguments, Hugging face args and LoRa args

In [19]:
YOUR_HF_USERNAME = 'Xuezha333'
project_name = "gemma-qlora-re"
current_timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
output_dir = f"../Checkpoints_{YOUR_HF_USERNAME}_Z/{project_name}_{current_timestamp}"

In [20]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim= "paged_adamw_32bit",
    save_steps=100,
    logging_steps=1,
    learning_rate=2e-4,
    max_grad_norm=0.3,
    
    max_steps=6000,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    gradient_checkpointing=True,
    fp16=False,
    bf16=False,
)

In [21]:

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

In [22]:
from peft import get_peft_model
m = get_peft_model(model, lora_config)
m.print_trainable_parameters()

trainable params: 25,001,984 || all params: 8,562,682,880 || trainable%: 0.29198773737606876


# 5.Formatting Function for model input

In [23]:
def formatting_func(example):
    text = f"### Question: {example['prompt']}\n### Answer: {example['completion']}"
    return text

# Train Model

In [24]:
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    peft_config=lora_config,
    tokenizer=tokenizer,
    max_seq_length=256,

    eval_dataset=eval_dataset,
    train_dataset=train_dataset, 

    formatting_func=formatting_func,
    packing=True,
    #dataset_text_field="text",
    neftune_noise_alpha=5,

)

trainer.train()

/root/miniconda3/envs/gemma/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:256: UserWarning: You passed a `neftune_noise_alpha` argument to the SFTTrainer, the value you passed will override the one in the `TrainingArguments`.
  warnings.warn(
Generating train split: 1702 examples [00:00, 3111.44 examples/s]
Generating train split: 390 examples [00:00, 3226.20 examples/s]
/root/miniconda3/envs/gemma/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:317: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/root/miniconda3/envs/gemma/lib/python3.8/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispa

Step,Training Loss
1,40.137500
2,27.443300
3,27.735600
4,30.842600
5,24.584400
6,25.431900
7,26.680700
8,27.282500
9,26.816400
10,27.745300


/root/miniconda3/envs/gemma/lib/python3.8/site-packages/peft/utils/other.py:581: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 3cfe8bad-fb28-4581-8d11-5eb52ce54727)') - silently ignoring the lookup for the file config.json in google/gemma-7b.
  warnings.warn(
/root/miniconda3/envs/gemma/lib/python3.8/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in google/gemma-7b - will assume that the vocabulary was not modified.
  warnings.warn(
/root/miniconda3/envs/gemma/lib/python3.8/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=Fals

TrainOutput(global_step=6000, training_loss=0.38549013455801956, metrics={'train_runtime': 20812.3566, 'train_samples_per_second': 1.153, 'train_steps_per_second': 0.288, 'total_flos': 2.8632927144247296e+17, 'train_loss': 0.38549013455801956, 'epoch': 14.08})